The goal of this notebook is to get heart rate activity data from a [Fitbit Heart Rate API endpoint](https://dev.fitbit.com/build/reference/web-api/heart-rate/).  The API endpoint format being used is


 and store the results into a csv file.

I am currently using the csv file in another notebook to retrieve the resting heart rate values.

# IMPORTS

In [43]:

import pandas as pd 
from dateutil import parser
from datetime import date
import json
import requests


# FUNCTIONS

In [44]:
def get_fitbit_formatted_date_from_user(start_end='start'):
    user_date = None
    print("Enter "+start_end+" date (or today for today's info):")
    user_date_input = input()
    if user_date_input.lower() == 'today':
        user_date = date.today()
    else:
        try:
            user_date = parser.parse(user_date_input)
        except ValueError as error:
            raise Exception(
                f'***** WE DID NOT RECEIVE A VALID DATE ... {error} *****')
    user_date_str = user_date.strftime('%Y-%m-%d')
    return user_date_str

In [45]:
def get_access_token():
    with open('config.json') as f:
        secret_dict = json.load(f)
    return secret_dict['ACCESS_TOKEN']

In [46]:
def get_results(access_token, start_date, end_date="7d"):
    # TODO: access token and start_date can't be none...
    api_endpoint = "https://api.fitbit.com/1/user/-/activities/heart/date/" + \
        start_date+"/"+end_date+".json"
    header = {'Authorization': 'Bearer {}'.format(access_token)}
    response = requests.get(api_endpoint, headers=header).json()
    # If the request was successful, there should be a KeyError exception when asking for the errors list.
    try:
        failed_response = response['errors'][0]
        raise Exception(failed_response['message'])
    except KeyError:
        pass  # it worked
    return response

# Code

## Get data from Fitbit

In [56]:
access_token = get_access_token()
start_date = get_fitbit_formatted_date_from_user('start')
end_date = get_fitbit_formatted_date_from_user('end')
data = get_results(access_token,start_date,end_date)

Enter start date (or today for today's info):
Enter end date (or today for today's info):


In [57]:

df = pd.DataFrame(data['activities-heart'][:][:])
df

,dateTime,value
0,2020-12-27,"{'customHeartRateZones': [], 'heartRateZones':..."
1,2020-12-28,"{'customHeartRateZones': [], 'heartRateZones':..."
2,2020-12-29,"{'customHeartRateZones': [], 'heartRateZones':..."
3,2020-12-30,"{'customHeartRateZones': [], 'heartRateZones':..."
4,2020-12-31,"{'customHeartRateZones': [], 'heartRateZones':..."
5,2021-01-01,"{'customHeartRateZones': [], 'heartRateZones':..."
6,2021-01-02,"{'customHeartRateZones': [], 'heartRateZones':..."
7,2021-01-03,"{'customHeartRateZones': [], 'heartRateZones':..."
8,2021-01-04,"{'customHeartRateZones': [], 'heartRateZones':..."


In [58]:
df.to_csv('resting_heartrate.csv',index=False)